### 1. Import libraries.

In [3]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from bs4 import BeautifulSoup
import configparser
import mysql.connector
import sqlalchemy
import plotly.express as px
import plotly.graph_objs as go
import json

### 2. Scrape.

In [59]:
#Open file containing rotating proxies.
try:
    proxies_file = open(r"C:\Users\jonat\data_projects\mentors_scraping\data\rotating_proxies_list.txt", "r")
except FileNotFoundError:
    proxies_file_path = r'C:\Users\jonat\data_projects\mentors_scraping\data\rotating_proxies_list.txt'
    print(f"Error: File not found at {proxies_file_path}")

#Set starting page number for scraping.
page = 1

#Create empty list to append scraped data to.
profiles_list = []

#Set up loop through proxies file to test and return first responsive proxy.
for proxy in proxies_file:

    #Send request to webpage.
    response = requests.get(url = f'https://mentorcruise.com/mentor/browse/?search=&sort=newest&tagsearch=&price__gt=0&price__lt=1200&tz=&type=&&page={page}', 
                            proxies = {'http': f"http://{proxy}"},
                            headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36",
                                        "Accept-Encoding": "gzip, deflate, br", 
                                        "Accept-Language": "en-US,en;q=0.5"}
                                        )
    
    #Set condition for responsive proxy.
    if response.status_code == 200:

        #Create BeautifulSoup object.    
        soup = BeautifulSoup(response.content, 'html.parser')

        #Return HTML structure for all profile data.
        profiles = soup.find_all('div', class_ = 'relative box px-7 py-8 transition-all duration-150 mb-12 max-w-screen-lg mx-auto')

        #Loop through HTML profile data.
        for profile in profiles:
            
            #Extract name.
            try:
                name = profile.select_one('h3', class_ = 'title text-2xl has-text-blue font-bold').text.split('\n')[1]
            except (IndexError, AttributeError, TypeError):
                name = None

            #Extract job_title.
            try:
                job_title = profile.select_one('span.has-text-blue.text-base.mt-2.inline-block').text.split('\n')[0]
            except (IndexError, AttributeError, TypeError):
                job_title = None
            
            #Extract company.
            try:
                company = profile.select_one('span.has-text-blue.text-base.mt-2.inline-block').text.split('\n')[1].split()[1]
            except (IndexError, AttributeError, TypeError):
                company = None

            #Extract rating.
            try:
                rating = profile.select_one('span.rating-display').text.split()[0]
            except (IndexError, AttributeError, TypeError):
                rating = None

            #Extract num_reviews.
            try:
                num_reviews = profile.select_one('span.rating-display').text.split()[1][1:]
            except (IndexError, AttributeError, TypeError):
                num_reviews = None
                
            #Extract long_description.
            try:   
                long_description = profile.select_one('div.break-word.mt-4.mb-6.max-w-screen-md.text-sm.leading-6').text.split('\n')[1]
            except (IndexError, AttributeError, TypeError):
                long_description = None

            #Extract dollars_month.
            try:
                dollars_month = profile.select_one('span.price-element.minimize').text.replace('$', '')
            except (IndexError, AttributeError, TypeError):
                dollars_month = None

            #Extract country.
            try:
                country = profile.select_one('div.relative.h-full.pb-16 span.text-xl.ml-1.align-middle')['title']
            except (IndexError, AttributeError, TypeError):
                country = None

            #Extract short_description.
            try:
                short_description = profile.select_one('div.relative.h-full.pb-16 span.font-medium').text.split('\n')[1]
            except (IndexError, AttributeError):
                short_description = None

            #Extract tags.
            try:
                services = profile.select_one('div.flex.space-x-4.cursor-default.inline-block.w-full.py-3.my-5.border-0.border-solid.border-t.border-b.border-gray-200.has-text-blue.text-sm.font-semibold')

                service_1 = services.text.split('\n')[5]
                service_2 = services.text.split('\n')[11] + ' Calls'
                service_3 = services.text.split('\n')[18]
                service_4 = services.text.split('\n')[24]

            except (IndexError, AttributeError, TypeError):
                service_1 = None
                service_2 = None
                service_3 = None
                service_4 = None

            #Append extracted data to dataframe.
            profiles_list.append(
                [name,
                job_title,
                company,
                rating, 
                num_reviews,
                dollars_month,
                country,
                short_description,
                long_description,
                service_1,
                service_2,
                service_3,
                service_4]
                ) 
            
        #Pause scraper to avoid having connection aborted by server. 
        time.sleep(5)

        #Increment page number by 1 to scrape next page.
        page += 1

    else:
        break

    #Assign condition to stop scraper at the very last page where the 'next' button is not found.
    if not soup.find('a', class_ = 'border-t-2 border-transparent pt-4 pl-1 inline-flex items-center text-sm font-medium text-gray-500 hover:text-gray-700 hover:border-gray-300'):
        break


### 3. Write to dataframe, rename columns, and re-format datatypes.

In [69]:
#Write to dataframe and rename columns.
profiles_df = pd.DataFrame(profiles_list, columns = ['name', 'job_title', 'company', 'rating', 'num_reviews', 'dollars_month', 'country', 'short_description', 'long_description', 'service_1', 'service_2', 'service_3', 'service_4'])

#Re-format datatypes.
profiles_df['rating'] = profiles_df['rating'].fillna(0).astype('float')
profiles_df['num_reviews'] = profiles_df['num_reviews'].fillna(0).astype('float')
profiles_df['dollars_month'] = profiles_df['dollars_month'].fillna(0).astype('int')

### 4. Write to CSV.

In [70]:
profiles_df.to_csv('..\data\mentors_profiles.csv', index = False)

### 5. Write to database.

In [71]:
#setup ini file and assign variables to authenticate connection.
config = configparser.ConfigParser()
config.read('../config.ini')

#access database authentication data.
host = config['mysql']['host']
user = config['mysql']['user']
password = config['mysql']['password']
database = config['mysql']['database']

#create an sqllchemy engine object to authenticate connection.
engine = sqlalchemy.create_engine(f'mysql+mysqlconnector://{user}:{password}@{host}/{database}')

#write the DataFrame to database.
profiles_df.to_sql(name='mentors_profiles', con=engine, if_exists='replace', index=False)

In [ ]:
--------------------------------